In [53]:
import nlpaug
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
ctx_aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 14.0kB/s]
c:\Kyode\clg\ml\ML_Project\venv\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gener\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 570/570 [00:00<00:00, 285kB/s]
Downloading: 100%

In [8]:
df_train = pd.read_csv("..\\Data\\PreprocessedData\\english_train_preprocess.csv")

In [9]:
df_train.head()

,Unnamed: 0,text,label,preprocessed_text
0,0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech,these tiktoks radiate gay chaotic energy and i...
1,1,@Champions Again He got killed for using false...,Non_hope_speech,<user> again he got killed for using false money
2,2,It's not that all lives don't matter,Non_hope_speech,its not that all lives dont matter
3,3,Is it really that difficult to understand? Bla...,Non_hope_speech,is it really that difficult to understand blac...
4,4,Whenever we say black isn't that racists? Why...,Non_hope_speech,whenever we say black isnt that racists why do...


In [11]:
df_train.groupby("label").size()

label
Hope_speech         1962
Non_hope_speech    20778
not-English           22
dtype: int64

In [17]:
hope = df_train[df_train.label=="Hope_speech"]["preprocessed_text"].values

In [18]:
hope

array(['network engineer here 23 and currently working as an instructor teaching men and women looking to be in it = next i want to teach at a university',
       'im still hiding my gender to my parents and they dont know im dating someone my friends knows already about my gender and they are very supportive of it and im very thankful to find true friends like them',
       'all lives matter without that we never have peace so to me forever all lives matter',
       ...,
       'i know that was so sweet of the boy to to say hope he knows is a good person',
       'i could maybe be sympathetic if they were less vague all i hear is them ascribing sinister motives to people who seem like they are just being overly friendly nerdsnand these testimonials right before the statstics on women engineers in silicon valley seem to imply that the reason that there are less women in tech there is because it is a hostile environment for women but step in your typical engineering classroom in america

In [21]:
x = []
for each in tqdm(hope):
    x.append(ctx_aug.augment(each))

100%|██████████| 1962/1962 [33:25<00:00,  1.02s/it] 


In [35]:
x = np.array(x).flatten()

In [27]:
syn_aug = naw.SynonymAug(aug_src='wordnet')

In [28]:
x2 = []
for each in tqdm(hope):
    x2.append(syn_aug.augment(each, n = 2))

100%|██████████| 1962/1962 [00:07<00:00, 272.79it/s]


In [36]:
x2 = np.array(x2).flatten()

In [38]:
x = np.append(x, x2)

In [98]:
x[-1], x[-2]

('this be inspiring im a mechanically skillful applied science student and i postulate this so bad',
 'this is inspiring im a mechanically skillful technology student and i postulate this thence bad')

In [76]:
x = x.astype(object)

In [87]:
x.shape

(5886,)

In [99]:
swap_aug = naw.RandomWordAug("swap", aug_max = 5)

In [117]:
x3 = []
for each in tqdm(x2):
    x3.append(swap_aug.augment(each))

100%|██████████| 3924/3924 [00:01<00:00, 3740.60it/s]


In [118]:
x3 = np.array(x3, dtype = object).flatten()

In [121]:
x3 = np.append(x3, x)

In [122]:
x3.shape

(9810,)

In [123]:
delete_aug = naw.RandomWordAug(aug_max = 3)

In [124]:
x4 = []
for each in tqdm(x3):
    x4.append(delete_aug.augment(each))

100%|██████████| 9810/9810 [00:01<00:00, 7643.13it/s]


In [125]:
x4 = np.array(x4).flatten()

In [126]:
x4.shape

(9810,)

In [127]:
x4 = np.append(x4, x3)

In [128]:
x4.shape

(19620,)

In [144]:
df_aug = pd.DataFrame({"augmented_text": x4, "label": "Hope_speech"})

In [146]:
df_train["augmented_text"] = df_train["preprocessed_text"]

In [149]:
df_aug = df_aug.append(df_train[["augmented_text", "label"]])

C:\Users\gener\AppData\Local\Temp\ipykernel_26816\2892258247.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aug = df_aug.append(df_train[["augmented_text", "label"]])


In [151]:
df_aug.shape

(42382, 2)

In [152]:
df_aug.to_csv("..//Data//AugmentedData//english_train_augmented.csv")